In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import numpy as np
import math
mpl.rcParams['figure.dpi']= 300
sns.set(rc={'text.usetex' : True})
import sys
sys.path.append("..")
import src.constants as cnst

In [ ]:
DATA_PATH = cnst.DATA_PATH
provided_data = cnst.provided_path
FIG_PATH = cnst.FIG_PATH

### Optimization Algorithm

In [ ]:
x = np.zeros(200) #x is an array of utilities
funds = 20000

In [ ]:
def geo_mean(x: np.ndarray) -> float:
    """
    Geometric mean with overflow protection
    """
    return np.exp(np.log(x).mean())

In [ ]:
def util(funds: float, green_pct: float, poverty_pct: float) -> float:
    """
    Calculate utility for a tract
    """
    return funds*(np.exp(poverty_pct + green_pct))

In [ ]:
def opt(x: np.ndarray, funds: int, injection: float) -> float:
    """
    Optimize the utility function for geographic tracts
    Returns geometric mean
    """
    while funds > 0:
        util_array = x
        total_util = geo_mean(util_array)
        max_util, max_util_idx = 0, 0
        for idx, u in enumerate(util_array):
            tract_utility = util(injection, green_pcts[idx], poverty_pcts[idx]))
            if cur_utility > max_util:
                max_util = tract_utility
                max_util_idx = idx
            utils[max_utils_idx] = max_util
            green_pcts[max_utils_idx] = new_green()
        
        funds -= injection

In [ ]:
df = pd.read_csv(DATA_PATH + provided_data)
df.rename(columns={'PercentPopIncomeBelow2xPovertyLevel': 'PctPvty', 'AvgReduxinNighttimeAnnualTemp_Celsius': 'TempRedux', 'Percent_GreenSpace': 'PctGrSpc'}, inplace=True)
green_pcts = df["PctGrSpc"].tolist()
poverty_pcts = df["PctPvty"].tolist()
